# Module 3: MCP (Model Context Protocol) Integration

**Duration:** ~30 minutes

**Learning Objectives:**
- Understand the Model Context Protocol architecture
- Connect ADK agents to existing MCP servers
- Build custom MCP servers with FastMCP
- Expose ADK tools as MCP servers
- Master MCP transport mechanisms (stdio, SSE, Streamable HTTP)

---

## 3.1 Understanding MCP

### What is MCP?

The **Model Context Protocol (MCP)** is an open standard (originally from Anthropic) that defines how AI applications connect to external data sources and tools. Think of it as a **universal adapter** that lets any AI agent work with any tool.

### The Problem MCP Solves

Before MCP, every AI application needed custom integrations:

```
BEFORE MCP (Fragmented)
────────────────────────

┌─────────┐     Custom API      ┌───────────┐
│ Agent A │────────────────────►│ Database  │
└─────────┘                     └───────────┘

┌─────────┐     Different API   ┌───────────┐
│ Agent B │────────────────────►│ Database  │
└─────────┘                     └───────────┘

┌─────────┐     Another API     ┌───────────┐
│ Agent C │────────────────────►│ Database  │
└─────────┘                     └───────────┘

Each agent needs a custom integration!
```

With MCP:

```
WITH MCP (Standardized)
────────────────────────

┌─────────┐                     ┌────────────┐     ┌───────────┐
│ Agent A │──┐                  │            │────►│ Database  │
└─────────┘  │                  │            │     └───────────┘
             │   MCP Protocol   │ MCP Server │
┌─────────┐  ├─────────────────►│            │────►┌───────────┐
│ Agent B │──┤                  │            │     │   Files   │
└─────────┘  │                  │            │     └───────────┘
             │                  │            │
┌─────────┐  │                  │            │────►┌───────────┐
│ Agent C │──┘                  │            │     │   APIs    │
└─────────┘                     └────────────┘     └───────────┘

One protocol, universal compatibility!
```

### MCP Architecture

MCP uses a **client-server** architecture:

| Component | Role | Example |
|-----------|------|--------|
| **MCP Host** | The AI application | ADK Runner |
| **MCP Client** | Connects to servers | McpToolset |
| **MCP Server** | Exposes tools/resources | Database connector |

### What MCP Servers Provide

1. **Tools**: Functions the agent can call
2. **Resources**: Data the agent can read (files, URLs)
3. **Prompts**: Reusable prompt templates

In [ ]:
# Setup
import os
from getpass import getpass

if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass("Enter your Gemini API Key: ")

# Install MCP-related packages if needed
!pip install -q mcp fastmcp httpx

print("✅ Setup complete!")

## 3.2 MCP Transport Mechanisms

MCP supports multiple transport mechanisms for client-server communication:

### 1. stdio (Standard I/O)
- Used for local MCP servers
- Server runs as a subprocess
- Communication via stdin/stdout

### 2. SSE (Server-Sent Events)
- For remote MCP servers over HTTP
- Uses two endpoints (POST for requests, GET for responses)
- Good for existing HTTP infrastructure

### 3. Streamable HTTP (Recommended)
- Newer transport protocol
- Single HTTP endpoint
- Supports streaming responses
- Production-ready

## 3.3 Connecting ADK to MCP Servers

ADK provides the `McpToolset` class to connect to MCP servers. Let's start with a simple example.

In [ ]:
from google.adk.agents import Agent
from google.adk.tools.mcp_tool import McpToolset, StdioServerParameters, SseServerParameters
from google.adk.runners import InMemoryRunner

print("✅ MCP imports successful!")

### Example: Connecting to a Filesystem MCP Server

The Filesystem MCP server is a common example that provides tools for reading/writing files.

In [ ]:
# Example: Creating an agent with MCP filesystem tools
# Note: This requires Node.js and the @anthropic/mcp-server-filesystem package

# For demonstration, here's how you would configure it:

filesystem_mcp_config = {
    "server_type": "stdio",
    "command": "npx",
    "args": ["-y", "@anthropic/mcp-server-filesystem", "/tmp"]
}

print("MCP Filesystem Server Configuration:")
print("=" * 50)
for key, value in filesystem_mcp_config.items():
    print(f"  {key}: {value}")

print("\n📝 Note: Actual connection requires Node.js installed")

In [ ]:
# Creating an agent with McpToolset
# This is the pattern you'll use for any MCP server

agent_with_mcp_template = '''
from google.adk.agents import Agent
from google.adk.tools.mcp_tool import McpToolset, StdioServerParameters

# Create the agent with MCP tools
agent_with_filesystem = Agent(
    name="file_assistant",
    model="gemini-2.0-flash",
    description="An assistant that can read and write files",
    instruction="""
    You are a file management assistant.
    Use the available tools to read, write, and manage files.
    """,
    tools=[
        McpToolset(
            connection_params=StdioServerParameters(
                command="npx",
                args=["-y", "@anthropic/mcp-server-filesystem", "/tmp"]
            )
        )
    ]
)
'''

print("📝 MCP Agent Template:")
print(agent_with_mcp_template)

## 3.4 Building Custom MCP Servers with FastMCP

**FastMCP** is a Python library that makes building MCP servers easy. Let's create a custom MCP server.

In [ ]:
from fastmcp import FastMCP
import json

# Create an MCP server for currency exchange
mcp_server_code = '''
from fastmcp import FastMCP
import httpx

# Initialize the MCP server
mcp = FastMCP("Currency Exchange Server")

@mcp.tool()
async def get_exchange_rate(
    source_currency: str,
    target_currency: str,
    amount: float = 1.0
) -> dict:
    """
    Get the current exchange rate and convert an amount.
    
    Args:
        source_currency: The source currency code (e.g., 'USD', 'EUR')
        target_currency: The target currency code (e.g., 'JPY', 'GBP')
        amount: The amount to convert (default 1.0)
    
    Returns:
        Exchange rate and converted amount
    """
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"https://api.frankfurter.app/latest",
                params={
                    "from": source_currency.upper(),
                    "to": target_currency.upper(),
                    "amount": amount
                },
                timeout=10.0
            )
            if response.status_code == 200:
                data = response.json()
                rate = data["rates"].get(target_currency.upper())
                return {
                    "status": "success",
                    "source": source_currency.upper(),
                    "target": target_currency.upper(),
                    "rate": rate / amount if amount != 0 else rate,
                    "amount": amount,
                    "converted": rate
                }
    except Exception as e:
        return {"status": "error", "message": str(e)}
    
    return {"status": "error", "message": "Unable to fetch exchange rate"}

@mcp.tool()
def list_currencies() -> dict:
    """
    List commonly supported currencies.
    
    Returns:
        Dictionary of currency codes and names
    """
    return {
        "currencies": {
            "USD": "US Dollar",
            "EUR": "Euro",
            "GBP": "British Pound",
            "JPY": "Japanese Yen",
            "CAD": "Canadian Dollar",
            "AUD": "Australian Dollar",
            "CHF": "Swiss Franc",
            "CNY": "Chinese Yuan"
        }
    }

# Run the server
if __name__ == "__main__":
    mcp.run()
'''

# Save the server code to a file
with open('/tmp/currency_mcp_server.py', 'w') as f:
    f.write(mcp_server_code)

print("✅ Currency MCP Server code saved to /tmp/currency_mcp_server.py")
print("\n📝 Server Code Preview:")
print("=" * 60)
print(mcp_server_code[:1500] + "...")

### Running the MCP Server

In a separate terminal, you would run:

```bash
python /tmp/currency_mcp_server.py
```

Or with uvicorn for HTTP transport:

```bash
uvicorn currency_mcp_server:mcp --host 0.0.0.0 --port 3000
```

## 3.5 Simulating MCP for Colab

Since Colab doesn't support running background processes easily, let's simulate the MCP pattern by creating equivalent ADK tools that match what an MCP server would provide.

In [ ]:
# Simulating MCP tools as ADK FunctionTools
# This demonstrates the same functionality an MCP server would provide

import httpx
from typing import Dict, Any
from google.adk.agents import Agent
from google.adk.tools import FunctionTool

async def get_exchange_rate(
    source_currency: str,
    target_currency: str,
    amount: float = 1.0
) -> Dict[str, Any]:
    """
    Get the current exchange rate and convert an amount.
    Uses the free Frankfurter API.
    
    Args:
        source_currency: The source currency code (e.g., 'USD', 'EUR')
        target_currency: The target currency code (e.g., 'JPY', 'GBP')
        amount: The amount to convert (default 1.0)
    
    Returns:
        Exchange rate and converted amount
    """
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"https://api.frankfurter.app/latest",
                params={
                    "from": source_currency.upper(),
                    "to": target_currency.upper(),
                    "amount": amount
                },
                timeout=10.0
            )
            if response.status_code == 200:
                data = response.json()
                converted = data["rates"].get(target_currency.upper())
                return {
                    "status": "success",
                    "source_currency": source_currency.upper(),
                    "target_currency": target_currency.upper(),
                    "exchange_rate": round(converted / amount, 4) if amount != 0 else converted,
                    "original_amount": amount,
                    "converted_amount": round(converted, 2),
                    "date": data.get("date")
                }
    except Exception as e:
        return {"status": "error", "message": str(e)}
    
    return {"status": "error", "message": "Unable to fetch exchange rate"}


def list_currencies() -> Dict[str, Any]:
    """
    List commonly supported currencies with their codes and names.
    
    Returns:
        Dictionary of currency codes and names
    """
    return {
        "status": "success",
        "currencies": {
            "USD": "US Dollar",
            "EUR": "Euro",
            "GBP": "British Pound Sterling",
            "JPY": "Japanese Yen",
            "CAD": "Canadian Dollar",
            "AUD": "Australian Dollar",
            "CHF": "Swiss Franc",
            "CNY": "Chinese Yuan",
            "INR": "Indian Rupee",
            "MXN": "Mexican Peso",
            "BRL": "Brazilian Real",
            "KRW": "South Korean Won"
        }
    }

print("✅ Currency exchange tools defined!")

In [ ]:
# Test the exchange rate function directly

result = await get_exchange_rate("USD", "EUR", 100)
print("💱 Direct API Test:")
print(f"   {result}")

In [ ]:
# Create a currency agent (simulating MCP integration)

currency_agent = Agent(
    name="currency_assistant",
    model="gemini-2.0-flash",
    description="A currency exchange assistant",
    instruction="""
    You are a helpful currency exchange assistant.
    
    Your capabilities:
    1. Convert amounts between different currencies
    2. Get current exchange rates
    3. List available currencies
    
    Guidelines:
    - Always show both the rate and the converted amount
    - If a currency isn't recognized, list available currencies
    - Be precise with numbers (show 2 decimal places for money)
    """,
    tools=[
        FunctionTool(get_exchange_rate),
        FunctionTool(list_currencies)
    ]
)

print(f"✅ Currency Agent created with {len(currency_agent.tools)} tools")

In [ ]:
# Helper function for agent interactions
from google.adk.runners import InMemoryRunner

async def chat_with_agent(agent: Agent, message: str) -> str:
    runner = InMemoryRunner(agent=agent)
    session = await runner.session_service.create_session(
        app_name=agent.name,
        user_id="workshop_user"
    )
    
    response_text = ""
    response = await runner.run(
        user_id="workshop_user",
        session_id=session.id,
        new_message=message
    )
    
    async for event in response:
        if hasattr(event, 'content') and event.content:
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text
    
    return response_text

# Test the currency agent
response = await chat_with_agent(
    currency_agent,
    "How much is 500 US dollars in Japanese Yen?"
)

print("💱 Currency Agent Test")
print("=" * 50)
print(f"User: How much is 500 US dollars in Japanese Yen?")
print(f"\nAgent: {response}")

In [ ]:
# Test with another query
response = await chat_with_agent(
    currency_agent,
    "What currencies can you convert? And what's the current EUR to GBP rate?"
)

print("💱 Multi-Tool Query Test")
print("=" * 50)
print(f"User: What currencies can you convert? And what's the current EUR to GBP rate?")
print(f"\nAgent: {response}")

## 3.6 Exposing ADK Tools as MCP Servers

You can also go the other direction: expose ADK tools as an MCP server so other clients can use them.

In [ ]:
# Code template for exposing ADK tools via MCP

adk_to_mcp_template = '''
from mcp import Server
from mcp.types import Tool, TextContent
from google.adk.tools import FunctionTool
from google.adk.tools.mcp_tool.conversion_utils import adk_to_mcp_tool_type

# Your ADK tool functions
def calculate_compound_interest(
    principal: float,
    rate: float,
    years: int,
    compounds_per_year: int = 12
) -> dict:
    """
    Calculate compound interest.
    
    Args:
        principal: Initial investment amount
        rate: Annual interest rate (as percentage, e.g., 5 for 5%)
        years: Number of years
        compounds_per_year: Compounding frequency (default 12 for monthly)
    
    Returns:
        Final amount and interest earned
    """
    r = rate / 100
    n = compounds_per_year
    t = years
    
    amount = principal * (1 + r/n) ** (n*t)
    interest = amount - principal
    
    return {
        "principal": principal,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2),
        "rate": rate,
        "years": years
    }

# Create ADK FunctionTool
adk_tool = FunctionTool(calculate_compound_interest)

# Create MCP server
app = Server("financial-tools-server")

@app.list_tools()
async def list_tools():
    """List available tools via MCP"""
    # Convert ADK tool schema to MCP format
    mcp_tool = adk_to_mcp_tool_type(adk_tool)
    return [mcp_tool]

@app.call_tool()
async def call_tool(name: str, arguments: dict):
    """Handle tool calls"""
    if name == "calculate_compound_interest":
        result = calculate_compound_interest(**arguments)
        return [TextContent(type="text", text=str(result))]
    raise ValueError(f"Unknown tool: {name}")

# Run server
if __name__ == "__main__":
    import asyncio
    from mcp.server.stdio import stdio_server
    
    asyncio.run(stdio_server(app))
'''

print("📝 ADK to MCP Server Template:")
print("=" * 60)
print(adk_to_mcp_template)

## 3.7 MCP Toolbox for Databases

Google provides **MCP Toolbox for Databases**, a production-ready MCP server that connects agents to various databases:

### Supported Databases
- **BigQuery** (SQL execution, schema discovery, forecasting)
- **AlloyDB** (PostgreSQL-compatible)
- **Cloud SQL** (PostgreSQL, MySQL, SQL Server)
- **Spanner**
- **Firestore**

### Using MCP Toolbox

In [ ]:
# Configuration example for MCP Toolbox for Databases

toolbox_config = '''
# tools.yaml - MCP Toolbox configuration
sources:
  my-bigquery:
    kind: bigquery
    project: your-gcp-project
    location: us-central1

tools:
  query_sales:
    kind: bigquery-sql
    source: my-bigquery
    description: "Query sales data from BigQuery"
    statement: |
      SELECT 
        product_name,
        SUM(quantity) as total_quantity,
        SUM(revenue) as total_revenue
      FROM `project.dataset.sales`
      WHERE date BETWEEN @start_date AND @end_date
      GROUP BY product_name
      ORDER BY total_revenue DESC
      LIMIT @limit
    parameters:
      - name: start_date
        type: DATE
        description: Start date for the query
      - name: end_date
        type: DATE
        description: End date for the query
      - name: limit
        type: INT64
        description: Maximum results to return
'''

print("📊 MCP Toolbox Configuration Example:")
print("=" * 60)
print(toolbox_config)

In [ ]:
# Using MCP Toolbox with ADK

toolbox_adk_example = '''
from google.adk.agents import Agent
from google.adk.tools.mcp_tool import McpToolset, SseServerParameters

# Assuming MCP Toolbox is running at localhost:5000
data_analyst_agent = Agent(
    name="data_analyst",
    model="gemini-2.0-flash",
    description="An agent that analyzes business data",
    instruction="""
    You are a data analyst assistant. Use the available tools to:
    1. Query sales and business data
    2. Generate insights and summaries
    3. Answer business questions with data
    
    Always explain your findings clearly.
    """,
    tools=[
        McpToolset(
            connection_params=SseServerParameters(
                url="http://localhost:5000/mcp"
            )
        )
    ]
)
'''

print("📊 ADK + MCP Toolbox Example:")
print("=" * 60)
print(toolbox_adk_example)

## 3.8 MCP Best Practices

### When to Use MCP

✅ **Use MCP when:**
- You want tools to be reusable across multiple agents/frameworks
- You need to integrate with existing MCP ecosystems
- You want to expose tools to non-ADK clients
- You need language-agnostic tool sharing

❌ **Use FunctionTool when:**
- Tools are specific to your ADK agent
- You need tight integration with Python code
- You want simpler deployment (no separate server)
- Performance is critical (no network overhead)

### MCP Server Design

1. **Single Responsibility**: Each server should focus on one domain
2. **Clear Schemas**: Provide detailed parameter descriptions
3. **Error Handling**: Return structured error responses
4. **Authentication**: Use proper auth for production (OAuth, API keys)
5. **Rate Limiting**: Protect against abuse

## 3.9 Hands-On Exercise: Build a Weather MCP Server

Create an MCP server that provides weather tools.

In [ ]:
# Exercise: Complete this MCP server template

weather_mcp_exercise = '''
from fastmcp import FastMCP

mcp = FastMCP("Weather Server")

@mcp.tool()
async def get_weather(city: str, units: str = "metric") -> dict:
    """
    Get current weather for a city.
    
    Args:
        city: City name
        units: "metric" or "imperial"
    
    Returns:
        Weather data
    """
    # TODO: Implement weather lookup
    # You can use wttr.in API (free, no key needed):
    # https://wttr.in/{city}?format=j1
    pass

@mcp.tool()
async def get_forecast(city: str, days: int = 3) -> dict:
    """
    Get weather forecast for upcoming days.
    
    Args:
        city: City name
        days: Number of forecast days (1-5)
    
    Returns:
        Forecast data
    """
    # TODO: Implement forecast lookup
    pass

if __name__ == "__main__":
    mcp.run()
'''

print("📝 Exercise: Complete the Weather MCP Server")
print("=" * 60)
print(weather_mcp_exercise)

In [ ]:
# Solution (partial - for reference)

import httpx

async def get_weather_solution(city: str, units: str = "metric") -> dict:
    """
    Get current weather using wttr.in API.
    """
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"https://wttr.in/{city}?format=j1",
                timeout=10.0
            )
            if response.status_code == 200:
                data = response.json()
                current = data["current_condition"][0]
                
                temp = current["temp_C"] if units == "metric" else current["temp_F"]
                unit_label = "°C" if units == "metric" else "°F"
                
                return {
                    "status": "success",
                    "city": city,
                    "temperature": f"{temp}{unit_label}",
                    "condition": current["weatherDesc"][0]["value"],
                    "humidity": f"{current['humidity']}%",
                    "wind": f"{current['windspeedKmph']} km/h"
                }
    except Exception as e:
        return {"status": "error", "message": str(e)}
    
    return {"status": "error", "message": "Failed to fetch weather"}

# Test the solution
result = await get_weather_solution("London")
print("🌤️ Weather Solution Test:")
print(f"   {result}")

---

## Summary

In this module, you learned:

1. **MCP Architecture**: Client-server model for tool integration
2. **Transport Mechanisms**: stdio, SSE, and Streamable HTTP
3. **McpToolset**: Connecting ADK agents to MCP servers
4. **FastMCP**: Building custom MCP servers in Python
5. **MCP Toolbox**: Google's production database connector
6. **Best Practices**: When to use MCP vs FunctionTool

### Key Takeaways

- MCP standardizes tool access across different AI frameworks
- ADK's `McpToolset` makes it easy to consume MCP servers
- FastMCP simplifies building MCP servers in Python
- Use MCP for reusable, cross-platform tools

### Next Steps

Continue to **Module 4: A2A Protocol** to learn how to build agents that communicate with each other.

---

### Resources

- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [ADK MCP Documentation](https://google.github.io/adk-docs/mcp/)
- [FastMCP GitHub](https://github.com/jlowin/fastmcp)
- [MCP Toolbox for Databases](https://github.com/googleapis/genai-toolbox)